In [ ]:
# XGBoost script --> WIP

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *
from sklearn.linear_model import LogisticRegression

In [1]:
label_map = {   "Black-grass"               :0,
                "Charlock"                  :1,
                "Cleavers"                  :2,
                "Common Chickweed"          :3,
                "Common wheat"              :4,
                "Fat Hen"                   :5,
                "Loose Silky-bent"          :6,
                "Maize"                     :7,
                "Scentless Mayweed"         :8,
                "Shepherds Purse"           :9,
                "Small-flowered Cranesbill" :10,
                "Sugar beet"                :11}

dim = 256
x_test = []
df_test = pd.read_csv('/input/plant_seedlings_classification/sample_submission.csv')

# Prepare testing data
for f, species in tqdm(df_test.values, miniters=100):
    img = cv2.imread('/input/plant_seedlings_classification/test/{}'.format(f))
    x_test.append(cv2.resize(img, (dim, dim)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)
x_test = x_test /255.0


# Preparing training data for prediction
x_train = []
y_train = []

dirs = os.listdir("/input/plant_seedlings_classification/train/")
for k in tqdm(range(len(dirs))):    # Directory
    files = os.listdir("/input/plant_seedlings_classification/train/{}".format(dirs[k]))
    for f in range(len(files)):     # Files
        img = cv2.imread('/input/plant_seedlings_classification/train/{}/{}'.format(dirs[k], files[f]))
        targets = np.zeros(12)
        targets[label_map[dirs[k]]] = 1 
        x_train.append(cv2.resize(img, (dim, dim)))
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)

x_train = x_train/ 255.0

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
100%|██████████| 794/794 [00:07<00:00, 112.36it/s]


(794, 256, 256, 3)


In [3]:
# Define all models
base_model = Xception(input_shape=(dim, dim, 3), include_top=False, weights=None, pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model_x = Model(inputs=base_model.input, outputs=predictions)

base_model = InceptionV3(input_shape=(dim, dim, 3), include_top=False, weights=None, pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model_i = Model(inputs=base_model.input, outputs=predictions)

base_model = ResNet50(input_shape=(dim, dim, 3), include_top=False, weights=None, pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model_r = Model(inputs=base_model.input, outputs=predictions)

base_model = InceptionResNetV2(input_shape=(dim, dim, 3), include_top=False, weights=None, pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model_ir = Model(inputs=base_model.input, outputs=predictions)

In [4]:
model_x.load_weights('/input/weights_x.h5')
model_i.load_weights('/input/weights_i.h5')
model_r.load_weights('/input/weights_r.h5')
model_ir.load_weights('/input/weights_ir.h5')

def predict(model,x):
    predictions = model.predict(x, verbose = 1)
    return predictions

pred_train_x = predict(model_x , x_train)
pred_train_i = predict(model_i , x_train)
pred_train_ir = predict(model_ir, x_train)
pred_train_r = predict(model_r, x_train)

pred_test_x = predict(model_x , x_test)
pred_test_i = predict(model_i , x_test)
pred_test_ir = predict(model_ir, x_test)
pred_test_r = predict(model_r, x_test)

794/794 [==============================] - 13s 16ms/step


In [5]:
# Get new features
Y = (y_train * range(12)).sum(axis=1)
X = np.hstack([pred_train_x, pred_train_i,pred_train_ir, pred_train_r])
T = np.hstack([pred_test_x, pred_test_i, pred_test_ir,pred_test_r])

from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X, Y)

# prefict on test set
y_pred = model.predict(T)

In [ ]:
from keras.utils import to_categorical
encoded = to_categorical(y_pred)

# funciton for submittable format
def get_submit(p_test):
    preds = []
    for i in range(len(p_test)):
        pos = np.argmax(p_test[i])
        preds.append(list(label_map.keys())[list(label_map.values()).index(pos)])
    return preds

# get submittable format
preds = get_submit(encoded)

df_test['species'] = preds
df_test.to_csv('/output/submission_111.csv', index=False)